In [7]:
#this notebook will do sentiment analysis on the disaster tweets from kaggle dataset using hugging face library

#importing libraries
import re
import torch

from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.nn import functional as F

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# initializing the model and tokenizer
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [9]:
#create custom dataset 

class disaster_tweets(Dataset):
    def __init__(self, train_text_list, labels, tokenizer):
        self.encoding = tokenizer(train_text_list, padding=True, truncation=True)
        self.labels = labels

        
    def __len__(self):
        return len(self.tweet)
    
    def __getitem__(self, idx):
        # return dictionary compatible with the huggingface transformers
        item = {key:torch.tensor(val[idx]) for key, val in self.encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [10]:
#load data from data/tweetclassified/train.csv and seperate into train and eval
train_csv_path = 'data/tweetclassified/train.csv'
df = pd.read_csv(train_csv_path)
df['text'] = df['text'].replace(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)',"",regex=True)
tweet = df['text'].tolist()
label = df['target'].tolist()

train_text, val_text, train_label, val_label = train_test_split(tweet, label, test_size=0.2, random_state=42)

train_dataset = disaster_tweets(train_text, train_label, tokenizer)
val_dataset = disaster_tweets(val_text, val_label, tokenizer)